In [19]:
import pandas as pd 
from backtesting import Strategy
from backtesting.lib import crossover
df = pd.read_csv("FUND_US_XNAS_VSMAX_9_8_2024_9_8_2025.csv")

# Covert Date to Date and times
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

# Set it as index
df.set_index("Date", inplace=True)

print(df.head())

              Open    High     Low   Close
Date                                      
2025-09-05  122.47  122.47  122.47  122.47
2025-09-04  121.61  121.61  121.61  121.61
2025-09-03  120.15  120.15  120.15  120.15
2025-09-02  120.33  120.33  120.33  120.33
2025-08-29  121.03  121.03  121.03  121.03


In [20]:
# Moving Average Crossover Strategy
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [24]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            

In [28]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

C:\Users\zuria\AppData\Local\Temp\ipykernel_2368\828152657.py:3: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
C:\Users\zuria\AppData\Local\Temp\ipykernel_2368\828152657.py:4: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2024-09-05 00:00:00
End                       2025-09-05 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    84.46215
Equity Final [$]                   7461.80708
Equity Peak [$]                       10000.0
Commissions [$]                     425.34252
Return [%]                          -25.38193
Buy & Hold Return [%]                  8.8719
Return (Ann.) [%]                   -25.46892
Volatility (Ann.) [%]                15.98842
CAGR [%]                            -18.30225
Sharpe Ratio                         -1.59296
Sortino Ratio                        -1.47177
Calmar Ratio                          -0.8085
Alpha [%]                           -24.83198
Beta                                 -0.06199
Max. Drawdown [%]                   -31.50152
Avg. Drawdown [%]                   -31.50152
Max. Drawdown Duration      326 days 00:00:00
Avg. Drawdown Duration      326 days 00:00:00
# Trades                          

In [29]:
bt.plot()

GridPlot(id='p1941', ...)

In [31]:
%%time

stats = bt.optimize(
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    maximize='Equity Final [$]',
    constraint=lambda param: param.n1 < param.n2,
)
stats

c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end 

CPU times: total: 1.86 s
Wall time: 1.96 s


Start                     2024-09-05 00:00:00
End                       2025-09-05 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    36.25498
Equity Final [$]                  12015.57454
Equity Peak [$]                   12015.57454
Commissions [$]                       38.1463
Return [%]                           20.15575
Buy & Hold Return [%]                -1.56727
Return (Ann.) [%]                    20.24368
Volatility (Ann.) [%]                19.87539
CAGR [%]                             13.51585
Sharpe Ratio                          1.01853
Sortino Ratio                         1.91991
Calmar Ratio                          1.45579
Alpha [%]                            19.53686
Beta                                 -0.39488
Max. Drawdown [%]                    -13.9056
Avg. Drawdown [%]                    -3.24071
Max. Drawdown Duration      150 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                          

In [32]:
stats._strategy

<Strategy SmaCross(n1=25,n2=60)>

In [33]:
bt.plot(plot_volume=False, plot_pl=False)

GridPlot(id='p2214', ...)

In [34]:
stats.tail()

SQN                                                              NaN
Kelly Criterion                                                  NaN
_strategy                                      SmaCross(n1=25,n2=60)
_equity_curve                       Equity  DrawdownPct DrawdownD...
_trades               Size  EntryBar  ExitBar  EntryPrice  ExitPr...
dtype: object

In [35]:
stats['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2024-09-05,10000.00000,0.000000,NaT
2024-09-06,10000.00000,0.000000,NaT
2024-09-09,10000.00000,0.000000,NaT
2024-09-10,10000.00000,0.000000,NaT
2024-09-11,10000.00000,0.000000,NaT
...,...,...,...
2025-08-29,11874.45454,0.007356,NaT
2025-09-02,11805.85454,0.013091,NaT
2025-09-03,11788.21454,0.014565,NaT
2025-09-04,11931.29454,0.002604,NaT


In [36]:
stats['_trades'] 

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,25)","Exit_SMA(C,25)","Entry_SMA(C,60)","Exit_SMA(C,60)"
0,-85,89,179,116.18,108.21,None,None,639.3037,38.1463,0.064738,2025-01-14,2025-05-23,129 days,None,117.8348,106.4692,118.465667,105.7355
